![CNN architecture](cnn_architecture_1.png)

In [1]:
import keras
from keras.datasets import mnist
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential

#Models in Keras can come in two forms – 
#Sequential and via the Functional API.
#allows you to easily stack sequential layers 
#(and even recurrent layers) of the network in order from input to output. 

model = Sequential()


Using TensorFlow backend.


In [2]:
# input image dimensions
img_x, img_y = 28, 28

# load the MNIST data set, which already splits into train and test sets for us
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# reshape the data into a 4D tensor - (sample_number, x_img_size, y_img_size, num_channels)
# because the MNIST is greyscale, we only have a single channel - RGB colour images would have 3
x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 1)
x_test = x_test.reshape(x_test.shape[0], img_x, img_y, 1)




In [3]:
# convert the data to the right type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
# convert class vectors to binary class matrices - this is for use in the
# categorical_crossentropy loss below
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
#add a 2D convolutional layer to process iput image
#32 is the number of output channels
#next input is the kernel_size, which in this case we have chosen to be a 5×5 moving window
#followed by the strides in the x and y directions (1, 1). 
#Next, the activation function is a rectified linear unit
#finally we have to supply the model with the size of the input to the layer


input_shape = (img_x, img_y, 1)

model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
#Next we add a 2D max pooling layer
#We simply specify the size of the pooling in the x and y directions 
#– (2, 2) in this case, and the strides.

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

In [8]:
#Next we add another convolutional 
#+ max pooling layer, with 64 output channels. 
#The default strides argument in the Conv2D() function is (1, 1) in Keras, so we can leave it out.
#The input tensor for this layer is (batch_size, 28, 28,  32) – 
#the 28 x 28 is the size of the image, 
#and the 32 is the number of output channels from the previous layer.
#However, notice we don’t have to explicitly detail what the shape of the input is – Keras will work it out for us
#This allows rapid assembling of network architectures 


model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))



In [9]:
#we want to flatten the output from these to enter our fully connected layers 
#The next two lines declare our fully connected layers – using the Dense() layer in Keras. 
#First we specify the size – in line with our architecture, we specify 1000 nodes, each activated by a ReLU function.  
#The second is our soft-max classification, or output layer, 
#which is the size of the number of our classes (10 in this case, for our 10 possible hand-written digits).

model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
#training and evaluating our CNN
#Keras supplies many loss functions
#In this case, we will use the standard cross entropy for categorical class classification 
batch_size = 100
epochs = 100
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

#now we train our model

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

#model evaluation
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Train on 60000 samples, validate on 10000 samples
Epoch 1/100
37300/60000 [=================>............] - ETA: 18s - loss: 1.1770 - accuracy: 0.7315